In [19]:
from voronoi.fortune import Voronoi
# import import_ipynb
# from delaunay import voronoi, delaunay
from voronoi.delaunay.delaunay import delaunay
from voronoi.delaunay.voronoi import voronoi
import numpy as np
from voronoi.utils import generate_uniform_points
import time
import os
import pandas as pd

In [20]:
TEST_SPEC = [(-1, 1, 5), (-1, 1, 10), (-1, 1, 20), (-1, 1, 35), (-1, 1, 50), (-1, 1, 75), (-1, 1, 100),
             (-5, 5, 150), (-5, 5, 200), (-5, 5, 300), (-5, 5, 400), (-10, 10, 500), (-20, 20, 1000)]
# (-50, 50, 1500), (-50, 50, 2000), (-50, 50, 3000), (-50, 50, 3500), (-50, 50, 4000), (-50, 50, 4500), (-100, 100, 5000)]
test_folder = 'tests'

In [21]:
def read_test_data(new=False):
    input_data = []
    folder = test_folder + '/in'
    if not os.path.exists(folder):
        os.makedirs(folder)
    for i, d in enumerate(TEST_SPEC):
        # if file of name test{i} exists, read it
        try:
            if new:
                raise Exception
            with open(f"{folder}/test{i}.in", "r") as f:
                lines = f.readlines()
                # Get the number of points from the first line
                num_points = int(lines[0])
                points = np.array([tuple(map(float, line.split()))
                                  for line in lines[1:]])
                input_data.append(points)
        except:
            with open(f"{folder}/test{i}.in", "w") as f:
                points = generate_uniform_points(*d)
                num_points = len(points)  # Get the number of points
                # Write the number of points at the start of the file
                f.write(f"{num_points}\n")
                f.write("\n".join([" ".join(map(str, p)) for p in points]))
                input_data.append(points)
    return input_data

In [22]:
def run_tests(saveOutput=True, tested=['fortune', 'delaunay']):
    test_data = read_test_data()
    times = {}
    # tests_n = 5
    tests_n = len(test_data)
    fortune = Voronoi()
    for method in tested:
        if not os.path.exists(f"{test_folder}/out/{method}"):
            os.makedirs(f"{test_folder}/out/{method}")
    for i in range(tests_n):
        points = test_data[i]
        for method in tested:
            print(f"Testing {method}, test #{i}, n={len(points)}...")
            start = time.time()
            if method == 'fortune':
                res = fortune.get_voronoi(points)
            elif method == 'delaunay':
                triang = delaunay(points)
                res = voronoi(triang, points)
            times[(i, method)] = time.time() - start
            print(f"Time elapsed: {times[(i, method)]}")
            if saveOutput:
                with open(f"{test_folder}/out/{method}/test{i}.out", "w") as f:
                    f.write("\n".join([" ".join(map(str, p)) for p in res]))
    return times

In [23]:
def save_and_visualize(new=True):
    if not new:
        df = pd.read_csv(f'{test_folder}/times.csv')
        n = df['n'].tolist()
        fortune = df['Fortune [s]'].tolist()
        delaunay = df['Delaunay [s]'].tolist()
    else:
        times = run_tests()
        fortune = [times[(i, 'fortune')]
                   for i, method in times if method == 'fortune']
        delaunay = [times[(i, 'delaunay')]
                    for i, method in times if method == 'delaunay']
        n = [TEST_SPEC[i][2] for i in range(len(delaunay))]

    data = []
    for i in range(len(n)):
        data.append([n[i], fortune[i], delaunay[i]])

    df = pd.DataFrame(
        data, columns=['n', 'Fortune [s]', 'Delaunay [s]'])
    df.plot(x='n', y=['Fortune [s]',
            'Delaunay [s]'], kind='line')
    if new:
        df.to_csv(f'{test_folder}/times.csv')
    return df

In [24]:
df = save_and_visualize(True)
df

Testing fortune, test #0, n=5...
Time elapsed: 0.11704730987548828
Testing delaunay, test #0, n=5...
Time elapsed: 0.004837751388549805
Testing fortune, test #1, n=10...


Time elapsed: 0.2615489959716797
Testing delaunay, test #1, n=10...
Time elapsed: 0.009349346160888672
Testing fortune, test #2, n=20...
Time elapsed: 0.5769846439361572
Testing delaunay, test #2, n=20...
Time elapsed: 0.01127934455871582
Testing fortune, test #3, n=35...
Time elapsed: 1.2058677673339844
Testing delaunay, test #3, n=35...
Time elapsed: 0.0331730842590332
Testing fortune, test #4, n=50...
Time elapsed: 2.6133852005004883
Testing delaunay, test #4, n=50...
Time elapsed: 0.040999412536621094
Testing fortune, test #5, n=75...
Time elapsed: 4.633727550506592
Testing delaunay, test #5, n=75...
Time elapsed: 0.08303451538085938
Testing fortune, test #6, n=100...
Time elapsed: 7.700474262237549
Testing delaunay, test #6, n=100...
Time elapsed: 0.17835569381713867
Testing fortune, test #7, n=150...
Time elapsed: 13.257596969604492
Testing delaunay, test #7, n=150...
Time elapsed: 0.1524500846862793
Testing fortune, test #8, n=200...
Time elapsed: 20.23344922065735
Testing delau